In [117]:
# dependencies
import pandas as pd

In [139]:
# read excel files into dataframes

season = "2015_2016"

file_to_load = f"Resources/{season}.xlsx"
df = pd.read_excel(file_to_load)

df.head()

,Player,FG,FGA,FG%,3P,FT,FTA,FT%,TRB,AST,STL,BLK,TOV,PS/G,VORP
0,Quincy Acy\acyqu01,2.0,3.6,0.556,0.3,0.8,1.2,0.735,3.2,0.5,0.5,0.4,0.5,5.2,0.5
1,Jordan Adams\adamsjo01,1.0,3.0,0.333,0.0,1.5,2.5,0.600,1.0,1.5,1.5,0.0,1.0,3.5,0.0
2,Steven Adams\adamsst01,3.3,5.3,0.613,0.0,1.4,2.5,0.582,6.7,0.8,0.5,1.1,1.1,8.0,2.1
3,Arron Afflalo\afflaar01,5.0,11.3,0.443,1.3,1.5,1.8,0.840,3.7,2.0,0.4,0.1,1.2,12.8,-0.2
4,Alexis Ajinca\ajincal01,2.5,5.3,0.476,0.0,0.9,1.1,0.839,4.6,0.5,0.3,0.6,0.9,6.0,-0.5


In [140]:
# clean player names
df["Player"] = df["Player"].str.split("\\", n = 1, expand = True)
df.head()

,Player,FG,FGA,FG%,3P,FT,FTA,FT%,TRB,AST,STL,BLK,TOV,PS/G,VORP
0,Quincy Acy,2.0,3.6,0.556,0.3,0.8,1.2,0.735,3.2,0.5,0.5,0.4,0.5,5.2,0.5
1,Jordan Adams,1.0,3.0,0.333,0.0,1.5,2.5,0.600,1.0,1.5,1.5,0.0,1.0,3.5,0.0
2,Steven Adams,3.3,5.3,0.613,0.0,1.4,2.5,0.582,6.7,0.8,0.5,1.1,1.1,8.0,2.1
3,Arron Afflalo,5.0,11.3,0.443,1.3,1.5,1.8,0.840,3.7,2.0,0.4,0.1,1.2,12.8,-0.2
4,Alexis Ajinca,2.5,5.3,0.476,0.0,0.9,1.1,0.839,4.6,0.5,0.3,0.6,0.9,6.0,-0.5


In [141]:
# keep only total season stats for players with more than 1 team
df = df.drop_duplicates(subset=["Player"], keep = 'first')
df.head()

,Player,FG,FGA,FG%,3P,FT,FTA,FT%,TRB,AST,STL,BLK,TOV,PS/G,VORP
0,Quincy Acy,2.0,3.6,0.556,0.3,0.8,1.2,0.735,3.2,0.5,0.5,0.4,0.5,5.2,0.5
1,Jordan Adams,1.0,3.0,0.333,0.0,1.5,2.5,0.600,1.0,1.5,1.5,0.0,1.0,3.5,0.0
2,Steven Adams,3.3,5.3,0.613,0.0,1.4,2.5,0.582,6.7,0.8,0.5,1.1,1.1,8.0,2.1
3,Arron Afflalo,5.0,11.3,0.443,1.3,1.5,1.8,0.840,3.7,2.0,0.4,0.1,1.2,12.8,-0.2
4,Alexis Ajinca,2.5,5.3,0.476,0.0,0.9,1.1,0.839,4.6,0.5,0.3,0.6,0.9,6.0,-0.5


In [142]:
# enter roster size and number of teams in fantasy league
roster_size = 13
num_teams = 10
sample_size = roster_size*num_teams

In [143]:
top_players = df.sort_values("VORP", ascending = False).head(sample_size)
top_players.head()

,Player,FG,FGA,FG%,3P,FT,FTA,FT%,TRB,AST,STL,BLK,TOV,PS/G,VORP
116,Stephen Curry,10.2,20.2,0.504,5.1,4.6,5.1,0.908,5.4,6.7,2.1,0.2,3.3,30.1,9.8
553,Russell Westbrook,8.2,18.1,0.454,1.3,5.8,7.2,0.812,7.8,10.4,2.0,0.3,4.3,23.5,8.3
264,LeBron James,9.7,18.6,0.520,1.1,4.7,6.5,0.731,7.4,6.8,1.4,0.6,3.3,25.3,7.6
205,James Harden,8.7,19.7,0.439,2.9,8.8,10.2,0.860,6.1,7.5,1.7,0.6,4.6,29.0,6.9
137,Kevin Durant,9.7,19.2,0.505,2.6,6.2,6.9,0.898,8.2,5.0,1.0,1.2,3.5,28.2,6.4


In [144]:
# calculate adjusted percentages
top_players['adj_FG'] = (10*(top_players['FG'].sum() / sample_size) + top_players["FG"]) / (10*(top_players['FGA'].sum() / sample_size) + top_players["FGA"])
top_players['adj_FT'] = (10*(top_players['FT'].sum() / sample_size) + top_players["FT"]) / (10*(top_players['FTA'].sum() / sample_size) + top_players["FTA"])
top_players.head()

,Player,FG,FGA,FG%,3P,FT,FTA,FT%,TRB,AST,STL,BLK,TOV,PS/G,VORP,adj_FG,adj_FT
116,Stephen Curry,10.2,20.2,0.504,5.1,4.6,5.1,0.908,5.4,6.7,2.1,0.2,3.3,30.1,9.8,0.473262,0.785853
553,Russell Westbrook,8.2,18.1,0.454,1.3,5.8,7.2,0.812,7.8,10.4,2.0,0.3,4.3,23.5,8.3,0.465485,0.775051
264,LeBron James,9.7,18.6,0.520,1.1,4.7,6.5,0.731,7.4,6.8,1.4,0.6,3.3,25.3,7.6,0.475243,0.761449
205,James Harden,8.7,19.7,0.439,2.9,8.8,10.2,0.860,6.1,7.5,1.7,0.6,4.6,29.0,6.9,0.463617,0.790153
137,Kevin Durant,9.7,19.2,0.505,2.6,6.2,6.9,0.898,8.2,5.0,1.0,1.2,3.5,28.2,6.4,0.473057,0.790335


In [145]:
# calculate z-scores and average z-score
top_players["zFG"] = (top_players["adj_FG"] - top_players["adj_FG"].mean()) / top_players["adj_FG"].std()
top_players["zFT"] = (top_players["adj_FT"] - top_players["adj_FT"].mean()) / top_players["adj_FT"].std()
top_players["z3P"] = (top_players["3P"] - top_players["3P"].mean()) / top_players["3P"].std()
top_players["zPTS"] = (top_players["PS/G"] - top_players["PS/G"].mean()) / top_players["PS/G"].std()
top_players["zREB"] = (top_players["TRB"] - top_players["TRB"].mean()) / top_players["TRB"].std()
top_players["zAST"] = (top_players["AST"] - top_players["AST"].mean()) / top_players["AST"].std()
top_players["zSTL"] = (top_players["STL"] - top_players["STL"].mean()) / top_players["STL"].std()
top_players["zBLK"] = (top_players["BLK"] - top_players["BLK"].mean()) / top_players["BLK"].std()
top_players["zTOV"] = (top_players["TOV"].mean() - top_players["TOV"]) / top_players["TOV"].std()
top_players["zAVG"] = (top_players["zFG"] + top_players["zFT"] + top_players["z3P"] + top_players["zPTS"] + top_players["zREB"] + top_players["zAST"] + top_players["zSTL"] + top_players["zBLK"] + top_players["zTOV"]) / 9
top_players.head()

,Player,FG,FGA,FG%,3P,FT,FTA,FT%,TRB,AST,...,zFG,zFT,z3P,zPTS,zREB,zAST,zSTL,zBLK,zTOV,zAVG
116,Stephen Curry,10.2,20.2,0.504,5.1,4.6,5.1,0.908,5.4,6.7,...,1.294292,1.394701,4.354377,2.825420,-0.197471,1.614538,2.342877,-0.904649,-1.789673,1.214935
553,Russell Westbrook,8.2,18.1,0.454,1.3,5.8,7.2,0.812,7.8,10.4,...,-0.467303,0.518237,0.243020,1.657801,0.724848,3.275344,2.115922,-0.733712,-2.989557,0.482733
264,LeBron James,9.7,18.6,0.520,1.1,4.7,6.5,0.731,7.4,6.8,...,1.742906,-0.585366,0.026632,1.976243,0.571128,1.659425,0.754190,-0.220903,-1.789673,0.459398
205,James Harden,8.7,19.7,0.439,2.9,8.8,10.2,0.860,6.1,7.5,...,-0.890630,1.743598,1.974117,2.630817,0.071539,1.973631,1.435056,-0.220903,-3.349522,0.596412
137,Kevin Durant,9.7,19.2,0.505,2.6,6.2,6.9,0.898,8.2,5.0,...,1.247873,1.758313,1.649537,2.489288,0.878567,0.851465,-0.153631,0.804716,-2.029650,0.832942


In [146]:
# rank by avg z-score
top_players = top_players.sort_values("zAVG", ascending = False).reset_index(drop=True)

In [147]:
top_players.index += 1

In [148]:
top_players

,Player,FG,FGA,FG%,3P,FT,FTA,FT%,TRB,AST,...,zFG,zFT,z3P,zPTS,zREB,zAST,zSTL,zBLK,zTOV,zAVG
1,Stephen Curry,10.2,20.2,0.504,5.1,4.6,5.1,0.908,5.4,6.7,...,1.294292,1.394701,4.354377,2.825420,-0.197471,1.614538,2.342877,-0.904649,-1.789673,1.214935
2,Kevin Durant,9.7,19.2,0.505,2.6,6.2,6.9,0.898,8.2,5.0,...,1.247873,1.758313,1.649537,2.489288,0.878567,0.851465,-0.153631,0.804716,-2.029650,0.832942
3,Kawhi Leonard,7.7,15.1,0.506,1.8,4.1,4.6,0.874,6.8,2.6,...,1.139497,1.172425,0.783988,1.250904,0.340548,-0.225814,1.662011,0.462843,0.370118,0.772947
4,Chris Paul,7.0,15.1,0.462,1.6,4.0,4.4,0.896,4.2,10.0,...,-0.115177,1.290574,0.567600,0.950153,-0.658630,3.095797,2.342877,-0.904649,-0.949754,0.624310
5,Anthony Davis,9.2,18.6,0.493,0.6,5.3,7.0,0.758,10.3,1.9,...,0.870861,-0.156502,-0.514336,1.799331,1.685596,-0.540021,0.527235,2.172208,-0.229824,0.623839
6,James Harden,8.7,19.7,0.439,2.9,8.8,10.2,0.860,6.1,7.5,...,-0.890630,1.743598,1.974117,2.630817,0.071539,1.973631,1.435056,-0.220903,-3.349522,0.596412
7,Hassan Whiteside,5.7,9.3,0.606,0.0,2.9,4.5,0.650,11.8,0.4,...,2.531468,-1.162083,-1.163497,0.012520,2.262045,-1.213320,-1.061453,5.078129,-0.109836,0.574886
8,Russell Westbrook,8.2,18.1,0.454,1.3,5.8,7.2,0.812,7.8,10.4,...,-0.467303,0.518237,0.243020,1.657801,0.724848,3.275344,2.115922,-0.733712,-2.989557,0.482733
9,Paul Millsap,6.2,13.2,0.470,0.9,3.8,5.0,0.757,9.0,3.3,...,0.043826,-0.087726,-0.189755,0.525565,1.186007,0.088392,1.662011,1.659399,-0.709777,0.464216
10,LeBron James,9.7,18.6,0.520,1.1,4.7,6.5,0.731,7.4,6.8,...,1.742906,-0.585366,0.026632,1.976243,0.571128,1.659425,0.754190,-0.220903,-1.789673,0.459398


In [149]:
top_players.to_excel(f"Resources/ranked_{season}.xlsx", index=True)